In [ ]:
import os
import os.path as osp
import mmcv
import torch
from mmcv.cnn.utils import revert_sync_batchnorm
from mmcv.runner import (get_dist_info, load_checkpoint, wrap_fp16_model)

from mmseg.apis import single_gpu_test
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import build_dp, get_device, setup_multi_processes

import pvt, pvtv2

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['LOCAL_RANK'] = '0'


In [3]:
cfg_path = '../my_configs/fpn_pvtv2_b5_casia512_320k.py'
checkpoint_path = '../work_dirs/casia512_320k/fpn_pvtv2_b5/iter_165000.pth'
cfg = mmcv.Config.fromfile(cfg_path)
opacity = 1
show = True
show_dir = '../work_dirs/casia512_320k/fpn_pvtv2_b5/preds'
eval_metrics = ['mIoU', 'mFscore', 'mDice']
cfg.work_dir = '../work_dirs/casia512_320k/fpn_pvtv2_b5'
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
json_file = osp.join(cfg.work_dir, f'eval_single_scale.json')

In [ ]:
setup_multi_processes(cfg)
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True
cfg.model.pretrained = None
cfg.data.test.test_mode = True
distributed = False
rank, _ = get_dist_info()
cfg.gpu_ids = range(1)
cfg.device = get_device()


In [ ]:
dataset = build_dataset(cfg.data.test)
loader_cfg = dict(
        num_gpus=len(cfg.gpu_ids),
        dist=distributed,
        shuffle=False)

loader_cfg.update({
        k: v
        for k, v in cfg.data.items() if k not in [
            'train', 'val', 'test', 'train_dataloader', 'val_dataloader',
            'test_dataloader'
        ]
})
test_loader_cfg = {
    **loader_cfg,
    'samples_per_gpu': 1,
    'shuffle': False,
    **cfg.data.get('test_dataloader', {})
}
data_loader = build_dataloader(dataset, **test_loader_cfg)

In [ ]:
cfg.model.train_cfg = None
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model.PALETTE = dataset.PALETTE

torch.cuda.empty_cache()
model = revert_sync_batchnorm(model)
model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)


In [ ]:
eval_kwargs = {}
results = single_gpu_test(
            model,
            data_loader,
            show,
            show_dir,
            False,
            opacity,
            pre_eval=eval_metrics,
            format_only=False,
            format_args=eval_kwargs)

eval_kwargs.update(metric=eval_metrics)
metric = dataset.evaluate(results, **eval_kwargs)
metric_dict = dict(config=cfg_path, metric=metric)
mmcv.dump(metric_dict, json_file, indent=4)
